# Chapter 5 - Resampling Methods

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


C:\Users\dmitr\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### 5. In Chapter 4, we used logistic regression to predict the probability of default using income and balance on the Default data set. We will now estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis.

In [4]:
default = pd.read_csv('./Datasets/Default.csv', index_col=0)
default_X = default[['income', 'balance']]
default_y = default['default']

In [5]:
default.student = default.student.map({'Yes': 1, 'No': 0})
default.default = default.default.map({'Yes': 1, 'No': 0})

(a) Fit a logistic regression model that uses income and balance to predict default 

In [6]:
d_lr_model = LogisticRegression()

(b) Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:  
i. Split the sample set into a training set and a validation set.



In [7]:
default_X_train, default_X_test, default_y_train, default_y_test = train_test_split(default_X, 
                                                                                    default_y,
                                                                                    test_size=0.2,
                                                                                    random_state=777)

ii. Fit a multiple logistic regression model using only the training observations.


In [8]:
d_lr_model.fit(default_X_train, default_y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.


In [9]:
default_cv_prediction = d_lr_model.predict(default_X_test)

iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

In [10]:
print('Validation set error is {:.2%}'.format(1-d_lr_model.score(default_X_test, default_y_test)))

Validation set error is 3.75%


(c) Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.

In [11]:
default_test_errors = []
for _ in range(3):
    default_X_train, default_X_test, default_y_train, default_y_test = train_test_split(default_X, 
                                                                                    default_y,
                                                                                    test_size=0.2,
                                                                                    random_state=100*_)
    model = LogisticRegression()
    model.fit(default_X_train, default_y_train)
    test_error = 1 - model.score(default_X_test, default_y_test)
    default_test_errors.append(test_error)
print('Validation set errors for 3 different splits are {}'.format(default_test_errors))

Validation set errors for 3 different splits are [0.037000000000000033, 0.032499999999999973, 0.025000000000000022]


(d) Now consider a logistic regression model that predicts the probability of default using income , balance , and a dummy variable for student . Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.

In [12]:
default_X = default[default.columns.difference(['default'])]
default_X_train, default_X_test, default_y_train, default_y_test = train_test_split(default_X, 
                                                                                    default_y,
                                                                                    test_size=0.2,
                                                                                    random_state=777)
d_lr_model.fit(default_X_train, default_y_train)
print('Validation set error for model with student predictor is {:.2%}'.format(1-d_lr_model.score(default_X_test, default_y_test)))

Validation set error for model with student predictor is 3.75%


Including student dummy variable have no effect on test error given all else equal.

### 6. We continue to consider the use of a logistic regression model to predict the probability of default using income and balance on the Default data set. In particular, we will now compute estimates for the standard errors of the income and balance logistic regression coefficients in two different ways: (1) using the bootstrap, and (2) using the standard formula for computing the standard errors in the glm() function. Do not forget to set a random seed before beginning your analysis.

#### For this excercise I will use statsmodels package to have inference from model.

(a) Using the summary() and glm() functions, determine the estimated standard errors for the coefficients associated with income and balance in a multiple logistic regression model that uses both predictors.

In [13]:
d_lr_model_sm = smf.logit('default ~ income + balance', data=default).fit()

Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10


In [14]:
d_lr_model_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Mon, 12 Dec 2016   Pseudo R-squ.:                  0.4594
Time:                        21:36:00   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
                                        LLR p-value:                4.541e-292
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept    -11.5405      0.435    -26.544      0.000       -12.393   -10.688
income      2.081e-05   4.99e-06      4.174      0.000       1.1e-05  3.06e-05
balance        0.0056      0.000     24.835      0.000         0.005     0.006
==============================================================================

Possibly complete quasi-separation: A fraction 0.14 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

(b) Write a function, boot.fn() , that takes as input the Default data set as well as an index of the observations, and that outputs the coefficient estimates for income and balance in the multiple logistic regression model.

In [15]:
def boot_fn(data, obs_ind):
    model = smf.logit('default ~ income + balance', data=data.ix[obs_ind]).fit(disp=False)
    return model.bse[1:]

In [16]:
def boot(data, bn=100):
    pars = [0, 0]
    n = len(data)
    for _ in range(bn):
        obs_ind = np.random.choice(data.index, size=n)
        pars += boot_fn(data, obs_ind)

    pars /= bn
    return pars

(c) Use the boot() function together with your boot.fn() function to estimate the standard errors of the logistic regression coefficients for income and balance.

In [20]:
print('Std. errors predicted with logistic regression model:\n {}, std. errors predicted with bootstrap:\n {}'.format(d_lr_model_sm.bse[1:], boot(default)))

Std. errors predicted with logistic regression model:
 income     0.000005
balance    0.000227
dtype: float64, std. errors predicted with bootstrap:
 income     0.000005
balance    0.000229
dtype: float64


(d) Comment on the estimated standard errors obtained using the glm() function and using your bootstrap function.  
Estimates are almost equal for both methods.

### 7. In Sections 5.3.2 and 5.3.3, we saw that the cv.glm() function can be used in order to compute the LOOCV test error estimate. Alternatively, one could compute those quantities using just the glm() and predict.glm() functions, and a for loop. You will now take this approach in order to compute the LOOCV error for a simple logistic regression model on the Weekly data set. Recall that in the context of classification problems, the LOOCV error is given in (5.4).

In [23]:
weekly = pd.read_csv('./Datasets/Weekly.csv', index_col=0)
weekly.Direction = weekly.Direction.map({'Up': 1, 'Down': 0})

(a) Fit a logistic regression model that predicts Direction using Lag1 and Lag2.